In [45]:
!pip install requests beautifulsoup4 pandas matplotlib -q

In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from typing import Dict, List, Optional
import re
from dataclasses import dataclass
from urllib.parse import urljoin
import time

## Подготовка данных для векторной базы данных

### HTML

In [47]:
URLS = {
    "ai": "https://abit.itmo.ru/program/master/ai",
    "ai_product": "https://abit.itmo.ru/program/master/ai_product"
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def fetch_page(url: str) -> Optional[str]:
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при загрузке {url}: {e}")
        return None

In [48]:
pages = {}
for name, url in URLS.items():
    html_content = fetch_page(url)
    if html_content:
        pages[name] = html_content
        print(f"Страница '{name}' успешно загружена ({len(html_content)} символов)")
    else:
        print(f"Не удалось загрузить страницу '{name}'")

Страница 'ai' успешно загружена (196405 символов)
Страница 'ai_product' успешно загружена (135602 символов)


In [49]:
ai_page = pages["ai"]
soup = BeautifulSoup(ai_page, 'html.parser')
for ai_data in soup(['style', 'script']):
  ai_data.decompose()
ai_text = ' '.join(soup.stripped_strings)

In [50]:
ai_text[:1000]

'Искусственный интеллект itmo.ru +7 (812) 480-0-480 abit@itmo.ru Eng Бакалавриат Магистратура Аспирантура Дополнительное образование ЛИЧНЫЙ КАБИНЕТ Бакалавриат Магистратура Основная информация Дополнительное образование Аспирантура Личный кабинет Образовательные программы магистратуры подать документы Искусственный интеллект институт прикладных компьютерных наук форма обучения очная длительность 2 года язык обучения русский общежитие да военный учебный центр да гос. аккредитация да дополнительные возможности СОП, Онлайн, Трек аспирантуры, ПИШ, Программа в сфере ИИ Менеджер программы Елизавета Витальевна Василенко aitalents@itmo.ru +7 (999) 526-79-88 Программа в соцсетях ВКонтакте Сайт Telegram направления подготовки 27.04.05 Инноватика 80 бюджетных 0 целевая 20 контрактных 09.04.01 Информатика и вычислительная техника 50 бюджетных 0 целевая 50 контрактных 11.04.02 Инфокоммуникационные технологии и системы связи 85 бюджетных 0 целевая 25 контрактных о программе Создавайте AI-продукты и 

In [51]:
ai_product_page = pages["ai_product"]
soup = BeautifulSoup(ai_product_page, 'html.parser')
for ai_product_data in soup(['style', 'script']):
  ai_product_data.decompose()
ai_product_text = ' '.join(soup.stripped_strings)

In [52]:
ai_product_text[:1000]

'Управление ИИ-продуктами / AI Product itmo.ru +7 (812) 480-0-480 abit@itmo.ru Eng Бакалавриат Магистратура Аспирантура Дополнительное образование ЛИЧНЫЙ КАБИНЕТ Бакалавриат Магистратура Основная информация Дополнительное образование Аспирантура Личный кабинет Образовательные программы магистратуры подать документы Управление ИИ-продуктами / AI Product институт прикладных компьютерных наук форма обучения очная длительность 2 года язык обучения русский общежитие да военный учебный центр да гос. аккредитация да дополнительные возможности СОП, Программа в сфере ИИ Менеджер программы Регина Ильдаровна Абдрашитова aiproduct@itmo.ru +7 (993) 639-86-77 Программа в соцсетях ВКонтакте Сайт Telegram направления подготовки 02.04.03 Математическое обеспечение и администрирование информационных систем 26 бюджетных 0 целевая 25 контрактных о программе Создавайте AI-продукты и технологии, которые меняют мир. Программа дает глубокие технические знания в области разработки систем искусственного интелле

### PDF

In [53]:
!pip install pdfplumber -q

In [57]:
import pandas as pd
import json
import re
from typing import Dict, List, Any
import os

import requests

def download_pdf(url, filename):
    try:
        if 'drive.google.com' in url:
            file_id = url.split('/d/')[1].split('/')[0]
            direct_url = f'https://drive.google.com/uc?export=download&id={file_id}'
        else:
            direct_url = url
        
        response = requests.get(direct_url)
        response.raise_for_status()
        
        with open(filename, 'wb') as f:
            f.write(response.content)
        
        print(f"✓ PDF файл скачан: {filename} ({len(response.content)} байт)")
        return True
    except Exception as e:
        print(f"✗ Ошибка загрузки PDF: {e}")
        return False

ai_pdf_url = "https://drive.google.com/file/d/1NSuBA0VkPzqNfdOD2IDR4TSDY2ea66nx/view"
ai_pdf_filename = "УП_Искусственный интеллект.pdf"
ai_product_pdf_url = "https://drive.google.com/file/d/1Uod9BdkrFqSfiCq6w2nLHQC-tPW_NNRJ/view"
ai_product_pdf_filename = "Учебный план AI Product.pdf"

download_pdf(ai_pdf_url, ai_pdf_filename)
download_pdf(ai_product_pdf_url, ai_product_pdf_filename)

✓ PDF файл скачан: УП_Искусственный интеллект.pdf (88123 байт)
✓ PDF файл скачан: Учебный план AI Product.pdf (79489 байт)


True

In [58]:
import pdfplumber
import pandas as pd
import json
import re
import os
from typing import Dict, List, Any

def extract_text_with_pdfplumber(pdf_path: str) -> Dict:    
    if not os.path.exists(pdf_path):
        print(f"Файл {pdf_path} не найден")
        return {}
    
    try:
        full_text = ""
        page_texts = []
        
        with pdfplumber.open(pdf_path) as pdf:
            print(f"✓ PDF открыт: {len(pdf.pages)} страниц")
            
            for page_num, page in enumerate(pdf.pages):
                text = page.extract_text()
                page_texts.append({
                    "page": page_num + 1,
                    "text": text
                })
                full_text += f"\n--- Страница {page_num + 1} ---\n{text}"
        
        return {
            "total_pages": len(pdf.pages),
            "full_text": full_text,
            "pages": page_texts,
            "text_length": len(full_text)
        }
    
    except Exception as e:
        print(f"Ошибка чтения PDF: {e}")
        return {}

def show_text_pdf_shortly(pdf_data):
    if pdf_data:
        print(f"Страниц: {pdf_data['total_pages']}")
        print(f"Символов: {pdf_data['text_length']}")
        
        print("\nПервые 1000 символов:")
        print("-" * 50)
        print(pdf_data['full_text'][:1000])
        print("-" * 50)

ai_pdf_data = extract_text_with_pdfplumber(ai_pdf_filename)
show_text_pdf_shortly(ai_pdf_data)
print('\n\n\n')
ai_product_pdf_data = extract_text_with_pdfplumber(ai_product_pdf_filename)
show_text_pdf_shortly(ai_product_pdf_data)

✓ PDF открыт: 5 страниц
Страниц: 5
Символов: 11760

Первые 1000 символов:
--------------------------------------------------

--- Страница 1 ---
Учебный план
ОП Искусственный интеллект
атратс
ыртсемеС Наименование модулей, дисциплин, практики и аттестации
де.з
в
ьтсокмеодурТ
.сач
в
ьтсокмеодурТ
Блок 1. Модули (дисциплины) 60 2160
Универсальная (надпрофессиональная) подготовка 12 432
Магистратура/Аспирантура ИИ 9 324
Мировоззренческий модуль + иняз 9 324
Иностранный язык 6 216
Иностранный язык 2 сем 3 108
2 Английский язык A1 / English A1 3 108
2 Английский язык A2 / English A2 3 108
2 Русский язык как иностранный / Russian as a foreign language 3 108
2 Английский язык в профессиональной деятельности / English for specific purposes 3 108
Иностранный язык 1 сем 3 108
1 Английский язык A1 / English A1 3 108
1 Английский язык A2 / English A2 3 108
1 Русский язык как иностранный / Russian as a foreign language 3 108
1 Английский язык в профессиональной деятельности / English for specific pu

### Сохранение для данных для векторной базы

In [59]:
import os
os.mkdir('../data')

FileExistsError: [WinError 183] Невозможно создать файл, так как он уже существует: '../data'

In [60]:
with open("../data/ai_program_info.txt", "w", encoding="utf-8") as f:
    f.write(ai_text)

with open("../data/ai_product_program_info.txt", "w", encoding="utf-8") as f:
    f.write(ai_product_text)

with open("../data/ai_study_plan.txt", "w", encoding="utf-8") as f:
    f.write(ai_pdf_data['full_text'])
    
with open("../data/ai_product_study_plan.txt", "w", encoding="utf-8") as f:
    f.write(ai_product_pdf_data['full_text'])

## Настройка LLM

In [ ]:
OPENAI_API_KEY = ""

In [61]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
!pip install llama-index


In [91]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
import asyncio
import os

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

2026-02-02 18:24:44,199 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [106]:
from llama_index.core import Settings

async def search_documents(query: str) -> str:
    response = await query_engine.aquery(query)
    return str(response)

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

agent = FunctionAgent(
    tools=[search_documents],
    llm=Settings.llm,
    system_prompt="""Ты ассистент, который помогает абитуриенту разобарться, какая из двух магистерских программ - 
    Искусственный интеллект и Управление ИИ-продуктами / AI Product,
    подходит ему лучше. В твоей базе есть содержимое html-страниц этих программ и учебные планы. 
    Умей такжее отвечать на вопросы по содержимому твоей базы. Твои ответы будут в телеграм боте, не используй markdown""",
)

### Тестирование

In [107]:
response = await agent.run(
        "Расскажи какие у тебя есть программы"
    )
print(response)

2026-02-02 18:29:01,039 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:29:01,552 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:29:01,629 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:29:04,766 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:29:07,483 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:29:08,469 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


У нас есть две магистерские программы:

1. **Искусственный интеллект**: Эта программа охватывает создание AI-продуктов и технологий, которые меняют мир. Студенты могут работать над проектами с ведущими компаниями, изучать различные роли, такие как ML Engineer, Data Engineer, AI Product Developer и Data Analyst, а также заниматься научной деятельностью, включая публикации и участие в международных конференциях. Обучение проходит в вечернее время, что позволяет совмещать его с работой.

2. **Управление ИИ-продуктами**: Эта программа включает создание AI-продуктов и технологий, которые могут изменить мир. Она предоставляет глубокие технические знания в области разработки систем искусственного интеллекта и навыки продуктового менеджмента. Студенты могут разрабатывать инновационные ИИ-решения и выводить их на рынок, а также участвовать в реальных проектах для крупных компаний. Выпускники могут претендовать на различные позиции, такие как AI Product Manager или AI Project Manager, с доходом 

In [108]:
response = await agent.run(
        "Хочу уметь создавать микросервисы. Подскажи на какую программу пойти лучше"
    )
print(response)

2026-02-02 18:29:18,013 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:29:18,430 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:29:18,477 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:29:22,239 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:29:22,762 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:29:25,354 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Если вы хотите уметь создавать микросервисы, вам стоит обратить внимание на программу "Искусственный интеллект". Эта программа включает проектный подход, где студенты работают над реальными проектами с ведущими компаниями, что может помочь вам развить навыки разработки и интеграции микросервисов.

Программа "Управление ИИ-продуктами" также может быть полезной, так как она предоставляет глубокие технические знания в разработке систем искусственного интеллекта и навыки продуктового менеджмента. Однако, если ваша цель - именно создание микросервисов, то "Искусственный интеллект" может быть более подходящим выбором.


In [109]:
response = await agent.run(
        "Хочу уметь создавать микросервисы. Подскажи на какую программу пойти лучше изходя из учебного плана"
    )
print(response)

2026-02-02 18:29:32,848 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:29:33,493 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:29:33,497 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:29:37,677 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:29:47,661 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:29:51,065 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Если вы хотите уметь создавать микросервисы, вам стоит обратить внимание на программу "Управление ИИ-продуктами". 

Учебный план этой программы включает дисциплины, такие как "Продуктовые исследования" и "Воркшоп по созданию продукта на данных", которые могут быть полезны для разработки и управления ИИ-продуктами. Также в программе предусмотрены выборные дисциплины, такие как "Бизнес-анализ" и "Управление проектами в Data Science", что может помочь вам в управлении проектами и разработке микросервисов.

Программа "Искусственный интеллект" также интересна, но она больше ориентирована на теоретические аспекты и универсальную подготовку в области ИИ, что может быть менее актуально для практической разработки микросервисов.

Таким образом, для вашей цели лучше подойдет программа "Управление ИИ-продуктами".


In [110]:
response = await agent.run(
        "Я разработчик на java, есть небольшой опыт программирования на python и созданию ml сервисов. Хочу дальше развиваься в этой сфере"
    )
print(response)

2026-02-02 18:29:58,846 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Учитывая ваш опыт в Java и Python, а также интерес к созданию ML-сервисов, обе магистерские программы могут быть вам интересны. 

Программа "Искусственный интеллект" может углубить ваши знания в области алгоритмов, машинного обучения и нейронных сетей, что будет полезно для разработки более сложных ML-сервисов.

Программа "Управление ИИ-продуктами" может быть более ориентирована на аспекты управления проектами, продуктами и командами в сфере ИИ, что может быть полезно, если вы хотите развиваться в направлении управления проектами или продуктов в этой области.

Если вы хотите углубить технические навыки и знания в области ИИ, выбирайте программу "Искусственный интеллект". Если вас больше интересует управление проектами и продуктами в сфере ИИ, то "Управление ИИ-продуктами" будет более подходящей.

Если хотите, я могу предоставить больше информации о каждой из программ.


In [111]:
response = await agent.run(
        "Подскажи какую программу, мне лучше выбрать"
    )
print(response)

2026-02-02 18:30:05,394 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Чтобы помочь вам выбрать между программами "Искусственный интеллект" и "Управление ИИ-продуктами", мне нужно больше информации о ваших интересах и целях. 

1. Какой у вас опыт в области технологий и программирования?
2. Какие темы вас больше интересуют: теоретические аспекты ИИ или управление проектами и продуктами в этой области?
3. Какие карьерные цели вы перед собой ставите?

Пожалуйста, поделитесь этой информацией, и я смогу дать более точный совет.


In [112]:
from llama_index.core.workflow import Context
ctx = Context(agent)

In [113]:
response = await agent.run(
    "Привет, я разработчик на java, есть небольшой опыт программирования на python и созданию ml сервисов. Хочу дальше развиваься в этой сфере",
    ctx=ctx
)
print(response)

2026-02-02 18:30:11,350 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Привет! Это здорово, что у тебя есть опыт в Java и Python, а также в создании ML-сервисов. Учитывая твои интересы и опыт, давай разберемся, какая из магистерских программ - Искусственный интеллект или Управление ИИ-продуктами - может подойти тебе лучше. 

Ты можешь рассказать, какие аспекты программирования и работы с ИИ тебя интересуют больше? Например, ты хочешь углубиться в технические детали разработки ИИ или больше интересуешься управлением проектами и продуктами в этой области?


In [114]:
response = await agent.run(
    "Подскажи какую программу, мне лучше выбрать исходя из прошлого опыта и то, что мне нравится",
    ctx=ctx
)
print(response)

2026-02-02 18:30:17,474 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:30:18,927 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:30:19,206 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:30:23,829 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:30:24,608 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:30:25,766 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Исходя из твоего опыта и интересов, вот краткое описание обеих программ:

1. **Искусственный интеллект**:
   - Программа охватывает создание AI-продуктов и технологий, которые могут изменить мир.
   - Включает проектный подход, где студенты работают над реальными проектами с ведущими компаниями.
   - Выпускники могут занять позиции, такие как ML Engineer, Data Engineer, AI Product Developer или Data Analyst.
   - Обучение проходит на русском языке в вечернее время, что позволяет совмещать учебу с работой.

2. **Управление ИИ-продуктами**:
   - Программа предлагает глубокие технические знания в области разработки систем искусственного интеллекта и навыки продуктового менеджмента.
   - Длится 2 года, также ведется на русском языке и включает очные воркшопы и онлайн-лекции.
   - Студенты работают над реальными проектами и могут выбирать темы для выпускных работ.
   - Выпускники могут претендовать на позиции, такие как AI Product Manager, AI Project Manager и AI Product Data Analyst, с дох

In [115]:
response = await agent.run(
     "Хочу уметь создавать микросервисы. Подскажи на какую программу пойти лучше изходя из учебного плана",
    ctx=ctx
)
print(response)

2026-02-02 18:30:39,511 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:30:40,078 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:30:40,126 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-02 18:30:44,375 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:30:51,271 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 18:30:56,992 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Вот краткое сравнение учебных планов обеих программ с акцентом на создание микросервисов:

1. **Искусственный интеллект**:
   - Программа включает модули, которые охватывают широкий спектр тем, включая создание продуктов на основе данных.
   - Важные дисциплины: "Процессы и методологии разработки решений на основе ИИ", "Воркшоп по созданию продукта на данных".
   - Студенты получают практические навыки в разработке и управлении ИИ-продуктами, что может включать создание микросервисов.

2. **Управление ИИ-продуктами**:
   - Программа включает модули, такие как "Проектирование систем машинного обучения" и "Управление проектами в Data Science".
   - Важные дисциплины: "Продуктовые исследования" и "Воркшоп по созданию продукта на данных".
   - Студенты также изучают процессы разработки и управления ИИ-продуктами, что может быть полезно для создания микросервисов.

Если твоя основная цель - научиться создавать микросервисы, обе программы могут дать тебе необходимые знания и навыки. Однако п

In [116]:
response = await agent.run(
     "Разве нет таких же программ и на Искусственном интеллекте?",
    ctx=ctx
)
print(response)

2026-02-02 18:31:14,608 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Да, ты прав. Программа "Искусственный интеллект" также включает модули, которые охватывают создание продуктов на основе данных и разработку систем, что может включать создание микросервисов. 

Обе программы предлагают обучение в области разработки и управления ИИ-продуктами, но с разными акцентами:

- **Искусственный интеллект**: больше фокусируется на технических аспектах разработки, включая создание и оптимизацию алгоритмов, работу с данными и разработку ИИ-систем. Это может быть полезно, если ты хочешь углубиться в технические детали и научиться создавать микросервисы.

- **Управление ИИ-продуктами**: акцент на управлении проектами и продуктами, что включает в себя навыки работы с командами, планирования и реализации проектов. Это может быть полезно, если ты хочешь развиваться в направлении управления продуктами и проектами в области ИИ.

Таким образом, если ты хочешь сосредоточиться на технических аспектах и создании микросервисов, программа "Искусственный интеллект" может быть бол

In [117]:
response = await agent.run(
     "Кем можно работать после обучения?",
    ctx=ctx
)
print(response)

2026-02-02 18:31:34,867 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Вот кем можно работать после обучения по каждой из программ:

1. **Искусственный интеллект**:
   - **ML Engineer**: Специалист по машинному обучению, который разрабатывает и внедряет модели машинного обучения.
   - **Data Engineer**: Инженер данных, который отвечает за создание и поддержку инфраструктуры для обработки и хранения данных.
   - **AI Product Developer**: Разработчик ИИ-продуктов, который создает и оптимизирует продукты на основе ИИ.
   - **Data Analyst**: Аналитик данных, который анализирует данные и предоставляет инсайты для бизнеса.
   - **Research Scientist**: Научный сотрудник, занимающийся исследованиями в области ИИ и машинного обучения.

2. **Управление ИИ-продуктами**:
   - **AI Product Manager**: Менеджер продукта, который отвечает за стратегию и развитие ИИ-продуктов.
   - **AI Project Manager**: Менеджер проектов, который управляет проектами в области ИИ и координирует работу команд.
   - **AI Product Data Analyst**: Аналитик данных, который работает с данными, 

In [118]:
index.storage_context.persist(persist_dir="../storage")